Nous allons détailler les fonctions dans ce notebook.

# IMPORTS , FONCTIONS ET URL

In [24]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options # en vue de faire tourner le browser en headless
from pathlib import Path
import os
# from webdriver_manager.chrome import ChromeDriverManager
# import chromedriver_binary
import re
import csv
import time
regex_expression = "</span> #.* in Kindle Store " # utile pour regex
#DRIVER_PATH_chromedriver = str(Path('../chromedriver').resolve()) ancienne ligne qui fonctionnait

from amazoncaptcha import AmazonCaptcha #librairi pour passer l'image capthca d'amazon
from selenium.webdriver.common.by import By




In [25]:
urlcaptcha = "https://www.amazon.com/errors/validateCaptcha"

#attention sur PC DELL UBUNTU le Wagon, bien penser à lancer X launch , et à désactiver VPN !!

In [7]:
configuration = { "wagon":"/usr/bin/chromedriver/chromedriver" , 
                 "raspberry":"/usr/bin/chromedriver" ,
                "anaconda-PS-stock":""}

In [8]:
environnement = input('Entrez la configuration matérielle')

Entrez la configuration matérielleraspberry


In [26]:
chemin_access = configuration[environnement]

In [27]:
chemin_access

'/usr/bin/chromedriver'

In [28]:
DRIVER_PATH_chromedriver = str(Path(chemin_access).resolve()) # installation chrome driver sur usr/bin


chrome_options = Options()
chrome_options.add_argument("--headless")

browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver, options=chrome_options)

browser.get("http://www.google.com")

#browser.get("http://www.amazon.com") #ici chrome est norma
#browser.get("http://www.amazon.com" , options = chome_options) # avec cette option le chrome est headless

/tmp/ipykernel_3396/1983623464.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver, options=chrome_options)


In [44]:
browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver)
browser.get("https://www.amazon.com/errors/validateCaptcha")

/tmp/ipykernel_3396/2394049935.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver)


In [45]:
link = browser.find_element(By.XPATH,"//div[@class = 'a-row a-text-center']//img").get_attribute('src')
captcha = AmazonCaptcha.fromlink(link)
captcha_value = AmazonCaptcha.solve(captcha)
input_field = browser.find_element(By.ID, "captchacharacters").send_keys(captcha_value)
button = browser.find_element(By.CLASS_NAME,"a-button-text")
button.click()

In [39]:
captcha_value

'GRGUFG'

In [ ]:
input

In [21]:
print(image_div)

https://images-na.ssl-images-amazon.com/captcha/twhhswbk/Captcha_uopquzefat.jpg


In [15]:
liste_categorie['url'][0].split('&dc')[0].split('%')[-1]

'3A157631011'

In [16]:
def trouver_node_url(url):
    """fonction qui retourne un lien simplifié et que l'on peu incrémenter d'une catégorie"""
    node = url.split('&dc')[0].split('%')[-1]
    lien = 'https://www.amazon.com/s?i=digital-text&rh=n%' + node +'&fs=true&page=1'
    #browser.get(lien)
    return(lien)

def trouver_node_url_a_incrementer(url):
    """fonction qui retourne un url que l'on peu incrémenter simplement il n'y a plus qu'a adjoindre i et l'incrémenter à la fin"""
    node = url.split('&dc')[0].split('%')[-1]
    lien = 'https://www.amazon.com/s?i=digital-text&rh=n%' + node +'&fs=true&page='
    #browser.get(lien)
    return(lien)

def y_a_t_il_des_livres_dans_page(url): #fonction qui détermine s'il y a des livres dans la page
    """ retourne True s'il y a des livres dans la page et False s'il n'y en a pas"""
    browser.get(url) #navigue sur une page avec tous les liens de la liste
    time.sleep(5)
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    livres = soup.find_all('div' ,{ 'data-asin':True, 'data-component-type' : 's-search-result'} ) # trouve tous les livres de la pages
    if livres!=[]:
        return True
    else : 
        return False
    
    
def trouver_les_livres_dans_page(url_page_principale):
    """fonction qui retourne une liste de lien web des livres présents dans la page"""
    browser.get(url_page_principale)
    time.sleep(5)
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    livres = soup.find_all('a',class_ = 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    liens_livres=['https://www.amazon.com'+livre.get('href') for livre in livres]  
    #on utilise liste comprehension pour prendre l'élement href (lien hypertexte) 
    # et ajouter www.amazon.com/ pour avoir un url exploitable
    return(liens_livres)

def compter_nb_livre_dans_page(url_page_principale):
    """fonction qui retourne une liste de lien web des livres présents dans la page"""
    browser.get(url_page_principale)
    time.sleep(5)
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    livres = soup.find_all('a',class_ = 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    liens_livres=['https://www.amazon.com'+livre.get('href') for livre in livres]  
    #on utilise liste comprehension pour prendre l'élement href (lien hypertexte) 
    # et ajouter www.amazon.com/ pour avoir un url exploitable
    return(len(liens_livres))

def rang_livre(url_d_un_livre):
    """ fonction qui retour le rang des vente d'une livre """
    browser.get(url_d_un_livre)
    time.sleep(5)
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    if soup.find_all( {'span'  : 'a-list-item'}) is not None : 
        rang_livre = soup.find_all( {'span'  : 'a-list-item'})
        if len(str(re.compile(regex_expression).search(str(rang_livre))).split())>5 : 
            rang = int(str(re.compile(regex_expression).search(str(rang_livre))).split()[5].replace("#","").replace(",",""))
        else : rang = None
    else : rang = None
    return(rang)


def titre_livre(url_d_un_livre):
    """fonction qui retourne le titre du livre lorsqu'on visite la page du livre """
    browser.get(url_d_un_livre)
    time.sleep(5)
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
#     if soup.find('span',class_ = "a-size-extra-large celwidget").text == None: 
#         pass
#     else: 
    titre=soup.find('span',class_ = "a-size-extra-large celwidget").text.strip()
        
    return(titre)


def lien_livre(url_d_un_livre):
    """fonction qui retour le lien du livre  """
    return(url_d_un_livre)
    


In [17]:
tableau_principal = []
for i in range(0,2): # i = 0 #doit aller de 0 à 5136 il y a 5137 éléments
    ligne_categorie = liste_categorie.iloc[i].tolist()
    #prends un éléments du dataframe en liste
    # permet de convertir un élément du dataframe en liste
    for j in range(0,2):# doit aller de 1 à ce qu'il n'y ait plus de page mais on peut partir de 1 à 2 ou 3 au départ.
        page_livres_multi=trouver_node_url_a_incrementer(liste_categorie.iloc[i].tolist()[0]) + str(j)
        page_livres_multi
    # for livre in page_livres_multi:
    # trouver_les_livres_dans_page(page_livres_multi)  # faire un joli code utiliser que la fonction retourne une liste et balayer la liste
    
    # for k in range(1,10):
    #     trouver_les_livres_dans_page(page_livres_multi)[k]
    #     page_livre = trouver_les_livres_dans_page(page_livres_multi)[k]
    #     liste_livre = [titre_livre(page_livre) , rang_livre(page_livre), lien_livre(page_livre)]
    #     ligne_supp = ligne_categorie + liste_livre
    #     tableau_principal.append(ligne_supp)

        trouver_les_livres_dans_page(page_livres_multi)  # faire un joli code utiliser que la fonction retourne une liste et balayer la liste

        for livre in trouver_les_livres_dans_page(page_livres_multi):
            page_livre = livre
            liste_livre = [titre_livre(livre) , rang_livre(livre), lien_livre(livre)]
            ligne_supp = ligne_categorie + liste_livre
            tableau_principal.append(ligne_supp)
tableau_principal

KeyboardInterrupt: 

In [12]:
titres_colonnes = ["url_categorie","cat1","cat2","cat3","cat4","cat5","cat6","titre","rang","lien"]

In [13]:
df = pd.DataFrame(tableau_principal, columns=titres_colonnes)

NameError: name 'tableau_principal' is not defined

In [76]:
df

,url_categorie,cat1,cat2,cat3,cat4,cat5,cat6,titre,rang,lien
0,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,Architectural Digest at 100: A Century of Style,None,https://www.amazon.com/Architectural-Digest-10...
1,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,A Place of My Own: The Architecture of Daydreams,None,https://www.amazon.com/Place-My-Own-Architectu...
2,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,The Stoic way of Life: The Ultimate Guide of S...,None,https://www.amazon.com/Stoic-way-Life-Confiden...
3,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,The $12 Million Stuffed Shark: The Curious Eco...,None,https://www.amazon.com/Million-Stuffed-Shark-E...
4,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,Confessions Of an Art Addict,None,https://www.amazon.com/Confessions-Art-Addict-...
5,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,The Gardner Heist: The True Story of the World...,None,https://www.amazon.com/Gardner-Heist-Worlds-La...
6,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,Wright Sites: A Guide to Frank Lloyd Wright Pu...,None,https://www.amazon.com/Wright-Sites-Guide-Publ...
7,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,The Fellowship: The Untold Story of Frank Lloy...,None,https://www.amazon.com/Fellowship-Untold-Story...
8,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,Frank Lloyd Wright,None,https://www.amazon.com/Frank-Lloyd-Wright-Mery...
9,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,Death in a Prairie House: Frank Lloyd Wright a...,None,https://www.amazon.com/Death-Prairie-House-Tal...


In [76]:
df.to_csv('2023-11.csv')